### Get results from Wandb

In [1]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("draftrec/atari_pretrain")

summary_list, config_list, id_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    id_list.append(run.id)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "id": id_list,
    })

runs_df.to_csv("project.csv")

### Read runs

In [2]:
data_ = pd.read_csv('project.csv', index_col=0)
data_

,summary,config,id
0,{},{},15mq5wub
1,{},{},2g5nwizy
2,"{'t_act_loss': 1.0548882588148116, 'mean_grad_...","{'env': {'game': 'Hero', 'seed': 42, 'type': '...",19wivx2v
3,"{'reward_f1': 0.9805117776648024, 'd_obs_loss'...","{'env': {'game': 'CrazyClimber', 'seed': 42, '...",u4p9q5tb
4,"{'d_act_loss': 3.235356698513031, 'max_grad_no...","{'env': {'game': 'Jamesbond', 'seed': 42, 'typ...",3g3h9ovv
...,...,...,...
1219,"{'negative_sim': 0.04744461342226714, 'pos_neg...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",5zlpym7v
1220,"{'_timestamp': 1659280381, 'negative_sim': 0.5...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",3oh4f4sw
1221,"{'_runtime': 18375, '_timestamp': 1659280529, ...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",385p3sw7
1222,"{'_wandb': {'runtime': 17734}, '_runtime': 177...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",njf5gwab


### Filter based on conditions

In [3]:
group_name = 'baseline'
exp_name = 'hicat_traj'
model_path = '0/10/model.pth'

In [4]:
configs = data_['config']
indexs = []
for idx, config in enumerate(configs):
    cfg = eval(config)
    if len(cfg) == 0:
        continue
        
    run_exp_name = cfg['exp_name']
    run_group_name = cfg['group_name']

    # condition
    if run_exp_name == exp_name and run_group_name == group_name:
        if 'env' in cfg:
            indexs.append(idx)

In [5]:
data = data_.iloc[indexs]
envs = []
for config in data['config']:
    envs.append(eval(config)['env']['game'])
data['env'] = envs
print(len(data))
data

26


/tmp/ipykernel_249902/3806693401.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['env'] = envs


,summary,config,id,env
2,"{'t_act_loss': 1.0548882588148116, 'mean_grad_...","{'env': {'game': 'Hero', 'seed': 42, 'type': '...",19wivx2v,Hero
3,"{'reward_f1': 0.9805117776648024, 'd_obs_loss'...","{'env': {'game': 'CrazyClimber', 'seed': 42, '...",u4p9q5tb,CrazyClimber
4,"{'d_act_loss': 3.235356698513031, 'max_grad_no...","{'env': {'game': 'Jamesbond', 'seed': 42, 'typ...",3g3h9ovv,Jamesbond
5,"{'act_f1': 0.6488219254622409, 't_act_acc': 0....","{'env': {'game': 'Freeway', 'seed': 42, 'type'...",xm3qiggc,Freeway
6,"{'_runtime': 18293, 'd_act_acc': 0.05171406338...","{'env': {'game': 'Frostbite', 'seed': 42, 'typ...",ykubu78v,Frostbite
7,"{'t_obs_loss': 0.2658317275941372, 't_rew_loss...","{'env': {'game': 'Gopher', 'seed': 42, 'type':...",1i2acso0,Gopher
8,"{'epoch': 3, 'act_f1': 0.3605604922371634, 'd_...","{'env': {'game': 'DemonAttack', 'seed': 42, 't...",25retjty,DemonAttack
9,"{'d_act_acc': 0.04485156324133277, 'mean_grad_...","{'env': {'game': 'ChopperCommand', 'seed': 42,...",s3gkkesa,ChopperCommand
10,"{'t_rtg_loss': 0.0013698149579110224, 'best_me...","{'env': {'game': 'UpNDown', 'seed': 42, 'type'...",3nom53qg,UpNDown
11,"{'t_obs_loss': 0.1592715836475858, '_wandb': {...","{'env': {'game': 'Seaquest', 'seed': 42, 'type...",yj56zpuf,Seaquest


### 1. Restore Saved Models

In [6]:
import pathlib
base_path = str(pathlib.Path().resolve())

artifact_dict = {}
for run_id, env in zip(data['id'], data['env']):
    print(env, run_id)
    try:
        name = env + '/' + model_path 
        path = base_path + '/' + name
        wandb.restore(name, run_path="draftrec/atari_pretrain/" + run_id)
        artifact_dict[path] = name 
    except:
        continue

Hero 19wivx2v
CrazyClimber u4p9q5tb
Jamesbond 3g3h9ovv
Freeway xm3qiggc
Frostbite ykubu78v
Gopher 1i2acso0
DemonAttack 25retjty
ChopperCommand s3gkkesa
UpNDown 3nom53qg
Seaquest yj56zpuf
RoadRunner 2krai32u
MsPacman nwlrx8j8
Krull 120gl19g
Qbert 1kvr3bpx
Kangaroo 1rvvsk95
PrivateEye 1wnw4y6r
Pong 2exsxack
KungFuMaster 2t5qsmrx
BankHeist ass2bxfa
Alien 21eu4rbw
Amidar 29p83hfu
Asterix 1v56ydd8
Breakout 25nekvnv
Assault 2fa9et8p
BattleZone 2h3p8e63
Boxing 95mdsoos


### Save to artifact

In [7]:
wandb.init(project='atari_pretrain',
           entity='draftrec',
           group=exp_name,
           settings=wandb.Settings(start_method="thread"))  
artifact = wandb.Artifact(name=exp_name, type='model')

# save models
for path, name in artifact_dict.items():
    artifact.add_file(path, name=name)

wandb.run.finish_artifact(artifact)

wandb: Currently logged in as: joonleesky (draftrec). Use `wandb login --relogin` to force relogin
/opt/conda/lib/python3.8/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


### Remove artifact

In [8]:
import shutil
for env in data['env']:
    shutil.rmtree('./'+env, ignore_errors=True)

### 2. Generate json file

In [ ]:
def get_results_dict(data):
    results = []
    for idx in range(len(data)):
        row = data.iloc[idx]
        summary = eval(row['summary'])
        config = eval(row['config'])

        if 'env' not in config:
            continue

        game = config['env']['game']
        try:
            import pdb
            pdb.set_trace()
            score = summary['eval_mean_traj_game_scores']
        except:
            continue
        results.append([0, game, score, 0])

    return results
    
results = get_results_dict(data)

In [ ]:
data

In [18]:
import json

def generate_json_file(file_name, results):
    data = {}
    data['data'] = results
    path = file_name + '.json'
    with open(path, 'w') as json_file:
        json.dump(data, json_file)

In [19]:
generate_json_file('byol_impala', results)